# Data Cleaning for Thesis

## File and Column Abbreviations

| Abreviation | Description                |
|-------------|----------------------------|
| CP          | (Adjusted) Closing Prices  |
| CR          | Current (Debt) Ratio       |
| DER         | Debt to Equity Ratio       |
| FCF         | Free Cash Flow             |
| HP          | High Price                 |
| LP          | Low Price                  |
| OP          | Open Price                 |
| PBR         | Price to Book Ratio        |
| PER         | Price Earnings Ratio       |
| PM          | Profit Margin              |
| PSR         | Price to Sales Ratio       |
| ROA         | Return on Assets           |
| ROE         | Return on (Common) Equity  |
| VOL         | Volume                     |

In [1]:
import pandas as pd
import numpy as np
import glob
import os
from tqdm.auto import tqdm
from ta import add_volatility_ta, add_momentum_ta, add_volume_ta, add_trend_ta, add_all_ta_features

# pip install ta  -> technical analysis

from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 100)

from statsmodels.tsa.stattools import adfuller

import concurrent.futures
from functools import partial

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
tickers = pd.read_excel("../DATA/Tickers/Company_Tickers.xlsx", sheet_name="UNIQUE BLOOMBERG", usecols=[4])["Ticker HC"].str.split().str[0].tolist()

In [3]:
def get_financials(folder_path, tickers):
    """
    Utility function to read all Excel files in a folder into a single data frame.
    Expects all excel files to have the same format.

    Parameters
    ----------
    folder_path : str
        Path to the folder containing the Excel files.
    tickers : list
        List of tickers to use as column names.

    Returns
    -------
    pd.DataFrame
        Data frame containing all the data from the Excel files.
    """

    # Initialize an empty dictionary to store the data frames
    data_frames = {}

    # Find all Excel files in the folder
    file_paths = glob.glob(os.path.join(folder_path, "*.xlsx"))

    # Iterate over the file paths
    for file_path in tqdm(file_paths, desc=f"Reading files", total=len(file_paths)):
        # Extract the file name without the extension
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        
        # Read the Excel file into a data frame
        df = pd.read_excel(file_path, sheet_name="HC", header=1, na_values=["#N/A N/A", "#N/A Requesting Data..."], names=tickers)
        df = df.add_suffix(f"_{file_name}")
        
        # Store the data frame in the dictionary using the file name as the key
        data_frames[file_name] = df

    return pd.concat(data_frames.values(), axis=1)

## Daily Data

### Macroeconomic Indicators

In [4]:
day_macro_US_CA = pd.read_excel("../DATA/Daily/Macro/USCA_Macro.xlsx", sheet_name="HC", header=1, na_values=["#N/A N/A", "#N/A Requesting Data..."]).iloc[2:]
day_macro_US_CA.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6254 entries, 2 to 6255
Columns: 56 entries, Indicator: to EUCBCI Index
dtypes: float64(1), object(55)
memory usage: 2.7+ MB


In [5]:
day_macro_US = pd.read_excel("../DATA/Daily/Macro/US_Macro.xlsx", sheet_name="HC", header=1, na_values=["#N/A N/A", "#N/A Requesting Data..."]).iloc[2:]
day_macro_US.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5881 entries, 2 to 5882
Columns: 56 entries, Indicator: to EUCBCI Index
dtypes: object(56)
memory usage: 2.5+ MB


In [6]:
day_macro_US_CA.index = pd.to_datetime(day_macro_US_CA["Indicator:"].values)
day_macro_US.index = pd.to_datetime(day_macro_US["Indicator:"].values)

In [7]:
day_macro_US_CA = day_macro_US_CA.drop(columns=["Indicator:"])
day_macro_US_CA = day_macro_US_CA[~day_macro_US_CA.index.isna()]
day_macro_US_CA = day_macro_US_CA.loc["2000-01-04" : "2023-05-31"]
day_macro_US_CA = day_macro_US_CA.ffill()
day_macro_US_CA = day_macro_US_CA.loc[:, day_macro_US_CA.isna().mean() < 0.6]
day_macro_US_CA = day_macro_US_CA.fillna(0)

day_macro_US = day_macro_US.drop(columns=["Indicator:"])
day_macro_US = day_macro_US[~day_macro_US.index.isna()]
day_macro_US = day_macro_US.loc["2000-01-04" : "2023-05-31"]
day_macro_US = day_macro_US.ffill()
day_macro_US = day_macro_US.loc[:, day_macro_US.isna().mean() < 0.6]
day_macro_US = day_macro_US.fillna(0)

In [8]:
day_macro_US_CA.head()

,IVEY Index,CAEICAIR Index,CACAPUTL Index,CACOUSCO Index,GCAN10YR Index,GCAN2YR Index,OEOTKLAF Index,RRCACONT Index,SPTSX Index,MXWO Index,...,CAIPYOY Index,COSYNFRM Index,IMP1YOY% Index,CAWCWGCY Index,CDGGBE10 Index,CL1 Comdty,CRB CMDT Index,EHSLMP%Y Index,OUSTUS Index,EUCBCI Index
2000-01-04,0.0,232810,0.0,107.9,6.402,6.054,101.1179,0.0,8202.60,1378.31,...,2.34,0.0,5.3,0.0,0.0,25.55,226.71,0.51,85.70,90.2
2000-01-05,0.0,218890,86.1,108.0,6.476,6.135,101.2076,0.0,8119.42,1359.54,...,3.74,0.9,7.0,3.1,0.0,24.91,229.42,1.54,86.04,90.5
2000-01-06,0.0,218890,86.1,108.0,6.410,6.070,101.2076,0.0,8114.24,1344.06,...,3.74,0.9,7.0,3.1,0.0,24.78,228.18,1.54,86.04,90.5
2000-01-07,0.0,218890,86.1,108.0,6.406,6.076,101.2076,0.0,8429.42,1375.88,...,3.74,0.9,7.0,3.1,0.0,24.22,228.92,1.54,86.04,90.5
2000-01-10,0.0,218890,86.1,108.0,6.421,6.085,101.2076,0.0,8602.50,1391.11,...,3.74,0.9,7.0,3.1,0.0,24.67,228.70,1.54,86.04,90.5


In [9]:
day_macro_US.head()

,NAPMPMI Index,NAPMNMI Index,INJCJC4 Index,CPTICHNG Index,LEI TOTL Index,USGG10YR Index,USGG2YR Index,OEOTKLAF Index,RAILORGT Index,SPX Index,...,PPI YOY Index,COSYNFRM Index,IMP1YOY% Index,WAGETRND Index,USGGBE10 Index,CL1 Comdty,CRB CMDT Index,EHSLMP%Y Index,OUSTUS Index,EUCBCI Index
2000-01-04,57.8,56.9,278.25,82.5152,92.1,6.594,6.358,101.2076,332085,1402.11,...,2.9,0.9,7.0,100.44,2.1138,24.91,229.42,1.54,86.04,90.5
2000-01-05,57.8,56.9,278.25,82.5152,92.1,6.524,6.299,101.2076,332085,1403.45,...,2.9,0.9,7.0,100.44,2.1138,24.78,228.18,1.54,86.04,90.5
2000-01-06,57.8,56.9,278.25,82.5152,92.1,6.515,6.308,101.2076,332085,1441.47,...,2.9,0.9,7.0,100.44,2.1138,24.22,228.92,1.54,86.04,90.5
2000-01-07,57.8,56.9,284.75,82.5152,92.1,6.552,6.359,101.2076,325119,1457.60,...,2.9,0.9,7.0,100.44,2.1138,24.67,228.70,1.54,86.04,90.5
2000-01-10,57.8,56.9,284.75,82.5152,92.1,6.657,6.428,101.2076,325119,1438.56,...,2.9,0.9,7.0,100.44,2.1138,25.77,227.77,1.54,86.04,90.5


In [10]:
day_macro_US_CA.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5858 entries, 2000-01-04 to 2023-05-31
Columns: 51 entries, IVEY Index to EUCBCI Index
dtypes: float64(41), int64(10)
memory usage: 2.3 MB


In [11]:
day_macro_US.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5858 entries, 2000-01-04 to 2023-05-31
Columns: 54 entries, NAPMPMI Index to EUCBCI Index
dtypes: float64(45), int64(9)
memory usage: 2.5 MB


In [12]:
day_macro_US_CA.to_parquet("../DATA/Daily/Processed/day_data_macro_USCA.parquet", compression="gzip")
day_macro_US.to_parquet("../DATA/Daily/Processed/day_data_macro_US.parquet", compression="gzip")

### Financial Indicators

In [13]:
day_fin_data = get_financials(folder_path="../DATA/Daily/Financials", tickers=tickers)

Reading files:   0%|          | 0/14 [00:00<?, ?it/s]

In [14]:
day_fin = day_fin_data.loc["2000-01-04" : "2023-05-31"]

In [15]:
day_fin.isna().sum().sum()

18769399

In [16]:
# Fill NaN values with the previous value (this allows us to set NAN values to 0 before the first trading day while having no NAN in the time series)
day_fin = day_fin.ffill()

In [17]:
day_fin.isna().sum().sum()

17268180

In [18]:
# % missing price values for each ticker (filter for columns that end in _CP, _HP, _LP, _OP, _VOL)
day_sparse_prices = day_fin.filter(regex="(_CP|_HP|_LP|_OP|_VOL)").isna().sum() / len(day_fin)
# Get the ticker symbols for which more than 60% of the price data is missing (we want at least ~10 years of data)
day_sparse_tickers = day_sparse_prices[day_sparse_prices > 0.6].index.str.split("_").str[0].unique()
day_tickers_clean = [ticker for ticker in tickers if ticker not in day_sparse_tickers]

In [19]:
len(tickers), len(day_sparse_tickers), len(day_tickers_clean)

(553, 144, 409)

In [20]:
# Save as txt file
with open("../DATA/Tickers/day_tickers_clean.txt", "w") as f:
    for ticker in day_tickers_clean:
        f.write(f"{ticker}\n")

In [21]:
# Select the columns that start with the sparse ticker symbols
for ticker in day_sparse_tickers:
    day_fin.drop(day_fin.filter(regex=f"^{ticker}_").columns, axis=1, inplace=True)

In [22]:
(day_fin.filter(regex="(_CP|_HP|_LP|_OP|_VOL)").isna().sum() / len(day_fin) > 0.6).sum() # Check

0

In [23]:
# Set rows with no closing price to NAN to have a clean time series
for ticker in day_tickers_clean:
    ticker_df = day_fin.filter(regex=f"^{ticker}_")
    day_fin.loc[ticker_df[f"{ticker}_CP"].isna(), ticker_df.columns] = np.nan

### Technical Indicators

In [24]:
day_fin_tec = day_fin.copy()

In [25]:
technical_dfs = []

for ticker in tqdm(day_tickers_clean):
    price_data = day_fin_tec.copy()[[f"{ticker}_OP", f"{ticker}_HP", f"{ticker}_LP", f"{ticker}_CP", f"{ticker}_VOL"]]
    day_fin_tec[f"{ticker}_Is_Listed"] = np.where(day_fin_tec[f"{ticker}_CP"].isna(), 0, 1)
    technical_indicators = add_all_ta_features(df=price_data.copy(), open=f"{ticker}_OP", high=f"{ticker}_HP", low=f"{ticker}_LP", close=f"{ticker}_CP", volume=f"{ticker}_VOL", fillna=True, colprefix=f"{ticker}_")
    technical_dfs.append(technical_indicators.drop(columns=price_data.columns))

  0%|          | 0/409 [00:00<?, ?it/s]

In [26]:
# technical_dfs = []

# for ticker in tqdm(day_tickers_clean):
#     price_data = day_fin_tec.copy()[[f"{ticker}_OP", f"{ticker}_HP", f"{ticker}_LP", f"{ticker}_CP", f"{ticker}_VOL"]]
#     day_fin_tec[f"{ticker}_Is_Listed"] = np.where(day_fin_tec[f"{ticker}_CP"].isna(), 0, 1)
#     technical_indicators = add_volatility_ta(df=price_data.copy(), high=f"{ticker}_HP", low=f"{ticker}_LP", close=f"{ticker}_CP", fillna=True, colprefix=f"{ticker}_")
#     technical_indicators = add_momentum_ta(df=technical_indicators, high=f"{ticker}_HP", low=f"{ticker}_LP", close=f"{ticker}_CP", volume=f"{ticker}_VOL", fillna=True, colprefix=f"{ticker}_")
#     technical_indicators = add_volume_ta(df=technical_indicators, high=f"{ticker}_HP", low=f"{ticker}_LP", close=f"{ticker}_CP", volume=f"{ticker}_VOL", fillna=True, colprefix=f"{ticker}_")
#     technical_indicators = add_trend_ta(df=technical_indicators, high=f"{ticker}_HP", low=f"{ticker}_LP", close=f"{ticker}_CP", fillna=True, colprefix=f"{ticker}_")
#     technical_dfs.append(technical_indicators.drop(columns=price_data.columns))

In [27]:
day_fin_tec = pd.concat([day_fin_tec] + technical_dfs, axis=1).fillna(0)

In [28]:
day_fin_tec.isna().sum().sum()

0

In [29]:
np.isinf(day_fin_tec).sum().sum()

0

In [30]:
day_fin_tec.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5858 entries, 2000-01-04 to 2023-05-31
Columns: 41309 entries, SLF_CR to DXT_others_cr
dtypes: float64(40900), int64(409)
memory usage: 1.8 GB


In [31]:
day_fin_tec.columns.duplicated()

array([False, False, False, ..., False, False, False])

In [32]:
day_fin_tec.to_parquet("../DATA/Daily/Processed/day_data_fin_tec.parquet", compression="gzip")

## Monthly Data

### Macroeconomic Indicators

In [33]:
month_macro_US_CA = pd.read_excel("../DATA/Monthly/Macro/USCA_Macro.xlsx", sheet_name="HC", header=1, na_values=["#N/A N/A", "#N/A Requesting Data..."]).iloc[2:]
month_macro_US_CA.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 2 to 289
Columns: 56 entries, Indicator: to EUCBCI Index
dtypes: float64(1), object(55)
memory usage: 126.1+ KB


In [34]:
month_macro_US = pd.read_excel("../DATA/Monthly/Macro/US_Macro.xlsx", sheet_name="HC", header=1, na_values=["#N/A N/A", "#N/A Requesting Data..."]).iloc[2:]
month_macro_US.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 2 to 284
Columns: 56 entries, Indicator: to EUCBCI Index
dtypes: object(56)
memory usage: 123.9+ KB


In [35]:
month_macro_US_CA.index = pd.to_datetime(month_macro_US_CA["Indicator:"].values)
month_macro_US.index = pd.to_datetime(month_macro_US["Indicator:"].values)

In [36]:
month_macro_US_CA = month_macro_US_CA.drop(columns=["Indicator:"])
month_macro_US_CA = month_macro_US_CA[~month_macro_US_CA.index.isna()]
month_macro_US_CA = month_macro_US_CA.loc["2000-01-04" : "2023-05-31"]
month_macro_US_CA = month_macro_US_CA.ffill()
month_macro_US_CA = month_macro_US_CA.loc[:, month_macro_US_CA.isna().mean() < 0.6]
month_macro_US_CA = month_macro_US_CA.fillna(0)

month_macro_US = month_macro_US.drop(columns=["Indicator:"])
month_macro_US = month_macro_US[~month_macro_US.index.isna()]
month_macro_US = month_macro_US.loc["2000-01-04" : "2023-05-31"]
month_macro_US = month_macro_US.ffill()
month_macro_US = month_macro_US.loc[:, month_macro_US.isna().mean() < 0.6]
month_macro_US = month_macro_US.fillna(0)

monthly_index = month_macro_US_CA.index

In [37]:
month_macro_US_CA.head()

,IVEY Index,CAEICAIR Index,CACAPUTL Index,CACOUSCO Index,GCAN10YR Index,GCAN2YR Index,OEOTKLAF Index,RRCACONT Index,SPTSX Index,MXWO Index,...,CAIPYOY Index,COSYNFRM Index,IMP1YOY% Index,CAWCWGCY Index,CDGGBE10 Index,CL1 Comdty,CRB CMDT Index,EHSLMP%Y Index,OUSTUS Index,EUCBCI Index
2000-01-31,0.0,255390,86.1,108.2,6.538,6.273,101.2716,0.0,8481.11,1338.25,...,3.74,0.9,7.1,3.1,0.0,27.64,225.03,2.32,86.24,90.6
2000-02-29,0.0,150690,86.1,108.3,6.126,5.984,101.2993,0.0,9128.99,1340.58,...,5.30,0.9,9.3,3.1,0.0,30.43,220.68,4.49,86.45,90.9
2000-03-31,0.0,144590,85.7,108.4,5.922,5.936,101.2977,0.0,9462.39,1431.94,...,4.97,4.4,9.2,3.2,0.0,26.90,228.01,3.59,86.87,91.1
2000-04-30,0.0,130150,85.7,108.5,6.170,6.139,101.2694,0.0,9347.61,1370.11,...,4.97,4.4,6.6,3.2,0.0,25.74,227.37,4.10,87.21,91.4
2000-05-31,0.0,135980,85.7,108.5,6.031,6.195,101.2148,0.0,9251.99,1334.14,...,5.23,4.4,6.1,3.2,0.0,29.01,234.16,3.57,87.40,91.7


In [38]:
month_macro_US.head()

,NAPMPMI Index,NAPMNMI Index,INJCJC4 Index,CPTICHNG Index,LEI TOTL Index,USGG10YR Index,USGG2YR Index,OEOTKLAF Index,RAILORGT Index,SPX Index,...,PPI YOY Index,COSYNFRM Index,IMP1YOY% Index,WAGETRND Index,USGGBE10 Index,CL1 Comdty,CRB CMDT Index,EHSLMP%Y Index,OUSTUS Index,EUCBCI Index
2000-01-31,56.7,56.6,289.00,82.1714,92.8,6.665,6.591,101.2716,327092,1394.46,...,2.5,0.9,7.1,100.44,2.3663,27.64,225.03,2.32,86.24,90.6
2000-02-29,55.8,56.2,293.75,82.1694,92.2,6.409,6.521,101.2993,322049,1366.42,...,4.0,0.9,9.3,100.44,2.1226,30.43,220.68,4.49,86.45,90.9
2000-03-31,54.9,57.4,269.75,82.2095,93.1,6.004,6.479,101.2977,317745,1498.58,...,4.3,4.4,9.2,100.53,2.0041,26.90,228.01,3.59,86.87,91.1
2000-04-30,54.7,58.0,273.00,82.4575,93.3,6.212,6.676,101.2694,324550,1452.43,...,3.6,4.4,6.6,100.53,2.2460,25.74,227.37,4.10,87.21,91.4
2000-05-31,53.2,57.7,282.25,82.4056,92.4,6.272,6.672,101.2148,338185,1420.60,...,3.7,4.4,6.1,100.53,2.1023,29.01,234.16,3.57,87.40,91.7


In [39]:
month_macro_US_CA.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 281 entries, 2000-01-31 to 2023-05-31
Columns: 51 entries, IVEY Index to EUCBCI Index
dtypes: float64(41), int64(10)
memory usage: 114.2 KB


In [40]:
month_macro_US.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 281 entries, 2000-01-31 to 2023-05-31
Columns: 54 entries, NAPMPMI Index to EUCBCI Index
dtypes: float64(45), int64(9)
memory usage: 120.7 KB


In [41]:
month_macro_US_CA.to_parquet("../DATA/Monthly/Processed/month_data_macro_USCA.parquet", compression="gzip")
month_macro_US.to_parquet("../DATA/Monthly/Processed/month_data_macro_US.parquet", compression="gzip")

### Financial Indicators

In [42]:
month_fin_data = get_financials(folder_path="../DATA/Monthly/Financials", tickers=tickers)

Reading files:   0%|          | 0/14 [00:00<?, ?it/s]

In [43]:
month_fin = month_fin_data.loc["2000-01-04" : "2023-05-31"]

In [44]:
# Monthly data index not the same for some reason (fixin it)
# Only keep the last entry for each month for each year
month_fin = month_fin.groupby([month_fin.index.year, month_fin.index.month]).last()
# Convert back to dataframe
month_fin = month_fin.reset_index(drop=True)
# Create datetime index
month_fin.index = monthly_index

In [45]:
month_fin.isna().sum().sum()

1221510

In [46]:
# Fill NaN values with the previous value (this allows us to set NAN values to 0 before the first trading day while having no NAN in the time series)
month_fin = month_fin.ffill()

In [47]:
month_fin.isna().sum().sum()

809211

In [48]:
# % missing price values for each ticker (filter for columns that end in _CP, _HP, _LP, _OP, _VOL)
month_sparse_prices = month_fin.filter(regex="(_CP|_HP|_LP|_OP|_VOL)").isna().sum() / len(month_fin)
# Get the ticker symbols for which more than 60% of the price data is missing (we want at least ~10 years of data)
month_sparse_tickers = month_sparse_prices[month_sparse_prices > 0.6].index.str.split("_").str[0].unique()
month_tickers_clean = [ticker for ticker in tickers if ticker not in month_sparse_tickers]

In [49]:
len(tickers), len(month_sparse_tickers), len(month_tickers_clean)

(553, 144, 409)

In [50]:
# Save as txt file
with open("../DATA/Tickers/month_tickers_clean.txt", "w") as f:
    for ticker in month_tickers_clean:
        f.write(f"{ticker}\n")

In [51]:
# Select the columns that start with the sparse ticker symbols
for ticker in month_sparse_tickers:
    month_fin.drop(month_fin.filter(regex=f"^{ticker}_").columns, axis=1, inplace=True)

In [52]:
(month_fin.filter(regex="(_CP|_HP|_LP|_OP|_VOL)").isna().sum() / len(month_fin) > 0.6).sum() # Check

0

In [53]:
# Set rows with no closing price to NAN to have a clean time series
for ticker in month_tickers_clean:
    ticker_df = month_fin.filter(regex=f"^{ticker}_")
    month_fin.loc[ticker_df[f"{ticker}_CP"].isna(), ticker_df.columns] = np.nan

### Technical Indicators

In [54]:
month_fin_tec = month_fin.copy()

In [55]:
technical_dfs = []

for ticker in tqdm(month_tickers_clean):
    price_data = month_fin_tec.copy()[[f"{ticker}_OP", f"{ticker}_HP", f"{ticker}_LP", f"{ticker}_CP", f"{ticker}_VOL"]]
    month_fin_tec[f"{ticker}_Is_Listed"] = np.where(month_fin_tec[f"{ticker}_CP"].isna(), 0, 1)
    technical_indicators = add_all_ta_features(df=price_data.copy(), open=f"{ticker}_OP", high=f"{ticker}_HP", low=f"{ticker}_LP", close=f"{ticker}_CP", volume=f"{ticker}_VOL", fillna=True, colprefix=f"{ticker}_")
    technical_dfs.append(technical_indicators.drop(columns=price_data.columns))

  0%|          | 0/409 [00:00<?, ?it/s]

In [56]:
# technical_dfs = []

# for ticker in tqdm(month_tickers_clean):
#     price_data = month_fin_tec.copy()[[f"{ticker}_OP", f"{ticker}_HP", f"{ticker}_LP", f"{ticker}_CP", f"{ticker}_VOL"]]
#     month_fin_tec[f"{ticker}_Is_Listed"] = np.where(month_fin_tec[f"{ticker}_CP"].isna(), 0, 1)
#     technical_indicators = add_volatility_ta(df=price_data.copy(), high=f"{ticker}_HP", low=f"{ticker}_LP", close=f"{ticker}_CP", fillna=True, colprefix=f"{ticker}_")
#     technical_indicators = add_momentum_ta(df=technical_indicators, high=f"{ticker}_HP", low=f"{ticker}_LP", close=f"{ticker}_CP", volume=f"{ticker}_VOL", fillna=True, colprefix=f"{ticker}_")
#     technical_indicators = add_volume_ta(df=technical_indicators, high=f"{ticker}_HP", low=f"{ticker}_LP", close=f"{ticker}_CP", volume=f"{ticker}_VOL", fillna=True, colprefix=f"{ticker}_")
#     technical_indicators = add_trend_ta(df=technical_indicators, high=f"{ticker}_HP", low=f"{ticker}_LP", close=f"{ticker}_CP", fillna=True, colprefix=f"{ticker}_")
#     technical_dfs.append(technical_indicators.drop(columns=price_data.columns))

In [57]:
month_fin_tec = pd.concat([month_fin_tec] + technical_dfs, axis=1).fillna(0)

In [58]:
np.isinf(month_fin_tec).sum().sum()

0

In [59]:
month_fin_tec.isna().sum().sum()

0

In [60]:
month_fin_tec.head()

,SLF_CR,ENGH_CR,HR-U_CR,IVN_CR,WFG_CR,BN_CR,BLDP_CR,EFR_CR,SAP_CR,PPL_CR,...,DXT_momentum_ppo,DXT_momentum_ppo_signal,DXT_momentum_ppo_hist,DXT_momentum_pvo,DXT_momentum_pvo_signal,DXT_momentum_pvo_hist,DXT_momentum_kama,DXT_others_dr,DXT_others_dlr,DXT_others_cr
2000-01-31,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-02-29,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-03-31,0.0,6.1861,0.0,0.0,1.9094,0.0991,20.5441,0.0,1.6102,1.1159,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-04-30,0.0,6.1861,0.0,0.0,1.9094,0.0991,20.5441,0.0,1.6102,1.1159,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-05-31,0.0,6.1861,0.0,0.0,1.9094,0.0991,20.5441,0.0,1.6102,1.1159,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
month_fin_tec.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 281 entries, 2000-01-31 to 2023-05-31
Columns: 41309 entries, SLF_CR to DXT_others_cr
dtypes: float64(40900), int64(409)
memory usage: 88.6 MB


In [62]:
month_fin_tec.to_parquet("../DATA/Monthly/Processed/month_data_fin_tec.parquet", compression="gzip")